In [1]:
library(tidyverse)
library(tidycensus)
library(sf)

Warning message:
“‘timedatectl’ indicates the non-existent timezone name ‘n/a’”
Warning message:
“Your system is mis-configured: ‘/etc/localtime’ is not a symlink”
Warning message:
“It is strongly recommended to set envionment variable TZ to ‘Etc/UCT’ (or equivalent)”
── Attaching packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.5 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.3      ✔ forcats 0.5.2 
── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Linking to GEOS 3.9.1, GDAL 3.2.1, PROJ 6.1.1; sf_use_s2() is TRUE



In [15]:
msa_code = "C3562" # "C4726" = Hampton Roads, "C3562" = NYC
base_path = "/home/data/census/"
msa_name = "nyc"
path = paste0(base_path, msa_name, "/")
tract_path = "geo/tracts.geojson"
block_group_path = "geo/block_groups.geojson"
block_path = "geo/blocks.geojson"
acs_path =  "acs/acs_wide.csv"
# census_api_key({key}, install = TRUE)

In [13]:
all_msa <- read_csv("https://www.bls.gov/cew/classifications/areas/qcew-county-msa-csa-crosswalk-csv.csv") # National MSA to county crosswalk
selected_msa <- all_msa %>% filter(`MSA Code` == msa_code)
selected_msa_counties <- selected_msa %>% pull(`County Code`)
selected_msa_states <- selected_msa %>% separate(`County Title`, sep = ",", into = c("name", "state")) %>%  pull(state) %>% unique()

Rows: 3251 Columns: 7
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (7): County Code, County Title, MSA Code, MSA Title, MSA Type, CSA Code,...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
tract_boundaries <- get_acs(geography = "tract",
                            state = selected_msa_states,
                            year = 2020, 
                            cb = TRUE, 
                            geometry = T, 
                            variables = "B19013_001") %>% 
  mutate(county = substr(GEOID, 1,5)) %>%
  filter(county %in% selected_msa_counties) %>%
  select(-variable, -estimate, -moe)

tract_boundaries <- st_transform(tract_boundaries, "WGS84")


st_write(tract_boundaries, paste0(path, tract_path))

In [ ]:
block_group_boundaries <- get_acs(geography = "block group", 
                                  state = selected_msa_states, 
                                  year = 2020, 
                                  cb = TRUE, 
                                  geometry = T,
                                  variables = "B19013_001") %>% 
  mutate(county = substr(GEOID, 1,5)) %>%
  filter(county %in% selected_msa_counties) %>%
  select(-variable, -estimate, -moe)

block_group_boundaries <- st_transform(block_group_boundaries, "WGS84")

st_write(block_group_boundaries, paste0(path, block_group_path))

In [14]:
block_boundaries <- get_decennial(geography = "block", 
                                  state = selected_msa_states, 
                                  year = 2010, 
                                  cb = TRUE, 
                                  geometry = T,
                                  variables = "H001001") %>% 
  mutate(county = substr(GEOID, 1,5)) %>%
  filter(county %in% selected_msa_counties) %>%
  select(-variable, -value)

block_boundaries <- st_transform(block_boundaries, "WGS84")

st_write(block_boundaries, paste0(path, block_path))

Getting data from the 2010 decennial Census




  |======================================================================| 100%


ERROR: Error in paste0(path, block_path): object 'path' not found


In [ ]:
# Income/Employment Variables

income_employment_vars <- c(
  "total_pop" = "B01001_001",
  "pop_in_poverty" = "B17001_002"
  # Add more
)

# Age/Disability Variables

age_disability_vars <- c(
  "male_under_18" = "B05003_003",
  "female_under_18" = "B05003_014",
  "total_over_65" = "B16007_014"
  # Add more
)

# Minority/Language Variables

race_language_vars <- c(
  "white_nh_pop" = "B03002_003",
  "black_nh_pop" = "B03002_004",
  "asian_nh_pop" = "B03002_006",
  "hispanic_pop" = "B03002_012"
  # Add more
)

# Housing/Transportation Variables

housing_transport_vars <- c(
  "total_households" = "B08201_001",
  "households_no_car" = "B08201_002"
  # Add more
)


In [ ]:
acs_vars <- c(income_employment_vars, age_disability_vars, race_language_vars, housing_transport_vars)

acs_long <- get_acs(geography = "tract",
                            state = selected_msa_states,
                            year = 2020, 
                            geometry = F, 
                            variables = acs_vars) %>% 
  mutate(county = substr(GEOID, 1,5)) %>%
  filter(county %in% selected_msa_counties) 

acs_wide <- acs_long %>%
  pivot_wider(id_cols = c(1,2), names_from = "variable", values_from = "estimate")


In [ ]:
# Compute new variables

acs_wide <- acs_wide %>%
  mutate(total_under_18 = female_under_18 + male_under_18,
         )

write_csv(acs_wide, paste0(path, acs_path))